In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


In [11]:
# Question 1 Analysis
np.random.seed(42)
n = 1000

W = np.random.normal(0, 1, (1000,))
X = W + np.random.normal(0, 1, (1000,))
Z = np.random.normal(1, 0, (1000,))  # Z = 1 (constant)
Y = X + Z + W + np.random.normal(0, 1, (1000,))

# The true error term from the data generating process
true_error = np.random.normal(0, 1, n)

# But if we model Y ~ X + Z, then W becomes part of the error term
# Error term when modeling Y = β₀ + β₁X + β₂Z + ε
# The "error" would include W plus the true random error
error_term = Y - X - Z  # This includes W + true_error

# Calculate correlation between X and this error term
correlation_X_error = np.corrcoef(X, error_term)[0, 1]
print(f"Correlation between X and error term: {correlation_X_error:.3f}")

# Also check correlation between X and W
correlation_X_W = np.corrcoef(X, W)[0, 1]
print(f"Correlation between X and W: {correlation_X_W:.3f}")

Correlation between X and error term: 0.440
Correlation between X and W: 0.686


In [3]:
# Question 2 Analysis
# If Y = β₀ + β₁X + β₂Z + error_term
# Then error_term = W + true_random_error
# We want Corr(X, error_term) = Corr(X, W + noise)

# X = W + noise1, where noise1 ~ N(0,1)
# error_term = W + noise2, where noise2 ~ N(0,1)

# Corr(X, error_term) = Corr(W + noise1, W + noise2)
# = Cov(W + noise1, W + noise2) / (Std(X) * Std(error_term))
# = Cov(W, W) / (Std(X) * Std(error_term))
# = Var(W) / (Std(X) * Std(error_term))

# Var(W) = 1
# Var(X) = Var(W) + Var(noise1) = 1 + 1 = 2, so Std(X) = √2
# Var(error_term) = Var(W) + Var(noise2) = 1 + 1 = 2, so Std(error_term) = √2

# Therefore: Corr(X, error_term) = 1 / (√2 * √2) = 1/2 = 0.5

print("Theoretical correlation between X and error term: 0.5")

Theoretical correlation between X and error term: 0.5


In [4]:
# Question 2 - Multiple simulations to get more stable estimate
correlations = []
n_simulations = 1000

for i in range(n_simulations):
    W = np.random.normal(0, 1, 1000)
    X = W + np.random.normal(0, 1, 1000)
    Z = np.random.normal(1, 0, 1000)  # Z = 1 (constant)
    Y = X + Z + W + np.random.normal(0, 1, 1000)
    
    # Error term when modeling Y ~ X + Z (so W becomes part of error)
    error_term = Y - X - Z  # This is W + random_noise
    
    correlation = np.corrcoef(X, error_term)[0, 1]
    correlations.append(correlation)

average_correlation = np.mean(correlations)
print(f"Average correlation across {n_simulations} simulations: {average_correlation:.3f}")
print(f"Standard deviation: {np.std(correlations):.3f}")

Average correlation across 1000 simulations: 0.500
Standard deviation: 0.024


In [5]:

# Load the data
df = pd.read_csv('homework_7.1.csv')

print("Data overview:")
print(df.head())
print("\nData description:")
print(df.describe())

# The question asks about controlling for W by regressing Y on X and Z 
# at constant values of W: -1, 0, and 1

# Since we can't literally use constant values (would give only one data point),
# we need to subset the data to observations near these W values

w_values = [-1, 0, 1]
coefficients = []
tolerance = 0.1  # Use observations within ±0.1 of target W values

print(f"\nAnalyzing coefficient of X at different W values:")
print("W Value | Sample Size | X Coefficient")
print("-" * 40)

for w_val in w_values:
    # Subset data to observations near this W value
    mask = (df['W'] >= w_val - tolerance) & (df['W'] <= w_val + tolerance)
    subset_df = df[mask]
    
    if len(subset_df) > 10:  # Ensure we have enough data points
        # Regress Y on X and Z for this subset
        X_vars = subset_df[['X', 'Z']]
        y = subset_df['Y']
        
        model = LinearRegression()
        model.fit(X_vars, y)
        
        x_coefficient = model.coef_[0]  # Coefficient of X
        coefficients.append(x_coefficient)
        
        print(f"{w_val:7.1f} | {len(subset_df):11d} | {x_coefficient:12.4f}")
    else:
        print(f"{w_val:7.1f} | {len(subset_df):11d} | Not enough data")

print(f"\nCoefficients: {coefficients}")

# Analyze the trend
if len(coefficients) >= 3:
    print(f"\nTrend analysis:")
    print(f"Coefficient at W=-1: {coefficients[0]:.4f}")
    print(f"Coefficient at W=0:  {coefficients[1]:.4f}")
    print(f"Coefficient at W=1:  {coefficients[2]:.4f}")
    
    # Check if increasing, decreasing, or staying the same
    diff1 = coefficients[1] - coefficients[0]
    diff2 = coefficients[2] - coefficients[1]
    
    print(f"Change from W=-1 to W=0: {diff1:.4f}")
    print(f"Change from W=0 to W=1:  {diff2:.4f}")
    
    if abs(diff1) < 0.2 and abs(diff2) < 0.2:
        print("Pattern: Staying about the same")
    elif diff1 > 0 and diff2 > 0:
        print("Pattern: Increasing")
    elif diff1 < 0 and diff2 < 0:
        print("Pattern: Decreasing")
    else:
        print("Pattern: Mixed/unclear")

Data overview:
   Unnamed: 0         X         W         Z         Y
0           0  1.137055  1.221768  0.327829  1.944532
1           1 -0.112905  0.465835  0.599650  0.655514
2           2  2.077755  1.795414 -0.063393  5.934411
3           3  0.456373 -0.512159  1.177413 -0.188064
4           4 -1.012402  0.080002 -0.275697 -0.533775

Data description:
        Unnamed: 0             X             W             Z             Y
count  10000.00000  10000.000000  10000.000000  10000.000000  10000.000000
mean    4999.50000     -0.021965     -0.001364      0.011044      0.479223
std     2886.89568      1.430124      1.006911      0.985831      3.345664
min        0.00000     -5.484932     -3.675430     -3.512546     -7.416559
25%     2499.75000     -1.007841     -0.675183     -0.655389     -1.938402
50%     4999.50000     -0.015165     -0.005634      0.007613     -0.050854
75%     7499.25000      0.915812      0.670535      0.687495      2.227702
max     9999.00000      5.807050      4.08

In [6]:
def make_error(corr_const, num):
    err = []
    prev = np.random.normal(0, 1)
    for n in range(num):
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, 1)
        err.append(prev)
    return np.array(err)

# Simulation parameters
n_trials = 500
sample_size = 1000
corr_values = [0.2, 0.5, 0.8]

results = {}

for corr_const in corr_values:
    print(f"\nAnalyzing corr_const = {corr_const}")
    
    x_coefficients = []
    x_std_errors = []
    
    for trial in range(n_trials):
        # Generate data
        np.random.seed(trial)  # For reproducibility
        
        # Generate treatment X and outcome Y with correlated errors
        error_x = make_error(corr_const, sample_size)
        error_y = make_error(corr_const, sample_size)
        
        # Create a simple linear model: Y = β₀ + β₁X + ε_y, X = α₀ + ε_x
        # Let's assume true coefficient β₁ = 1 for X
        true_beta = 1.0
        true_alpha = 0.0
        
        X = true_alpha + error_x
        Y = true_beta * X + error_y
        
        # Add intercept term for regression
        X_matrix = np.column_stack([np.ones(sample_size), X])
        
        # Fit regression manually to get standard errors
        # β = (X'X)⁻¹X'Y
        XtX_inv = np.linalg.inv(X_matrix.T @ X_matrix)
        beta_hat = XtX_inv @ X_matrix.T @ Y
        
        # Calculate residuals and standard errors
        y_pred = X_matrix @ beta_hat
        residuals = Y - y_pred
        mse = np.sum(residuals**2) / (sample_size - 2)  # degrees of freedom
        
        # Standard errors: sqrt(diag(MSE * (X'X)⁻¹))
        se_beta = np.sqrt(mse * np.diag(XtX_inv))
        
        x_coefficients.append(beta_hat[1])  # Coefficient of X (not intercept)
        x_std_errors.append(se_beta[1])     # Standard error of X coefficient
    
    # Calculate statistics
    std_dev_estimates = np.std(x_coefficients)
    mean_std_errors = np.mean(x_std_errors)
    
    results[corr_const] = {
        'std_dev_estimates': std_dev_estimates,
        'mean_std_errors': mean_std_errors,
        'ratio': std_dev_estimates / mean_std_errors
    }
    
    print(f"Standard deviation of X coefficient estimates: {std_dev_estimates:.4f}")
    print(f"Mean of standard error estimates: {mean_std_errors:.4f}")
    print(f"Ratio (std_dev / mean_se): {std_dev_estimates / mean_std_errors:.4f}")

# Compare ratios across correlation values
print(f"\n{'Correlation':>12} | {'Std Dev':>8} | {'Mean SE':>8} | {'Ratio':>8}")
print("-" * 50)
for corr in corr_values:
    r = results[corr]
    print(f"{corr:>12.1f} | {r['std_dev_estimates']:>8.4f} | {r['mean_std_errors']:>8.4f} | {r['ratio']:>8.4f}")

# Analyze trend in ratios
ratios = [results[corr]['ratio'] for corr in corr_values]
print(f"\nRatio trend: {ratios}")

if ratios[2] / ratios[0] > 1.1:
    print("The ratio (i) / (ii) increases as correlation increases")
elif ratios[2] / ratios[0] < 0.9:
    print("The ratio (i) / (ii) decreases as correlation increases")
else:
    print("The ratio (i) / (ii) remains the same")


Analyzing corr_const = 0.2


Standard deviation of X coefficient estimates: 0.0338
Mean of standard error estimates: 0.0316
Ratio (std_dev / mean_se): 1.0695

Analyzing corr_const = 0.5
Standard deviation of X coefficient estimates: 0.0418
Mean of standard error estimates: 0.0316
Ratio (std_dev / mean_se): 1.3222

Analyzing corr_const = 0.8
Standard deviation of X coefficient estimates: 0.0684
Mean of standard error estimates: 0.0316
Ratio (std_dev / mean_se): 2.1654

 Correlation |  Std Dev |  Mean SE |    Ratio
--------------------------------------------------
         0.2 |   0.0338 |   0.0316 |   1.0695
         0.5 |   0.0418 |   0.0316 |   1.3222
         0.8 |   0.0684 |   0.0316 |   2.1654

Ratio trend: [np.float64(1.0695011843435516), np.float64(1.322245984700657), np.float64(2.165399212055331)]
The ratio (i) / (ii) increases as correlation increases


In [10]:
# Question 2 - Alternative interpretation
# If we model Y ~ X + Z without knowing about W
correlations = []
n_simulations = 1000

for i in range(n_simulations):
    W = np.random.normal(0, 1, (1000,))
    X = W + np.random.normal(0, 1, (1000,))
    Z = np.random.normal(1, 0, (1000,))  # Z = 1 (constant)
    Y = X + Z + W + np.random.normal(0, 1, (1000,))
    
    # If Z is constant (all 1's), then when we regress Y on X and Z,
    # the Z term just becomes part of the intercept
    # So effectively we're looking at Y - intercept vs X
    
    # Let's check if Z is actually constant
    if np.std(Z) < 1e-10:  # Z is constant
        # When Z is constant, regressing Y on X and Z is equivalent to
        # regressing Y on X with an intercept
        from sklearn.linear_model import LinearRegression
        
        model = LinearRegression()
        model.fit(X.reshape(-1, 1), Y)
        
        # The error term is the residual
        error_term = Y - model.predict(X.reshape(-1, 1))
        
        correlation = np.corrcoef(X, error_term)[0, 1]
        correlations.append(correlation)

average_correlation = np.mean(correlations)
print(f"Average correlation when Z is constant: {average_correlation:.6f}")
print(f"Standard deviation: {np.std(correlations):.6f}")

# Check what happens if Z is not constant
print("\nNow checking with non-constant Z:")
correlations_nonconst = []

for i in range(100):
    W = np.random.normal(0, 1, (1000,))
    X = W + np.random.normal(0, 1, (1000,))
    Z = np.random.normal(0, 1, (1000,))  # Now Z is not constant
    Y = X + Z + W + np.random.normal(0, 1, (1000,))
    
    # Regress Y on X and Z
    from sklearn.linear_model import LinearRegression
    
    model = LinearRegression()
    model.fit(np.column_stack([X, Z]), Y)
    
    error_term = Y - model.predict(np.column_stack([X, Z]))
    correlation = np.corrcoef(X, error_term)[0, 1]
    correlations_nonconst.append(correlation)

print(f"Average correlation when Z varies: {np.mean(correlations_nonconst):.6f}")

Average correlation when Z is constant: 0.000000
Standard deviation: 0.000000

Now checking with non-constant Z:
Average correlation when Z varies: -0.000000
